In [1]:
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/arbethi/diabetic-retinopathy-level-detection?select=preprocessed+dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sharanya22
Your Kaggle Key: ········


100%|█████████████████████████████████████████████████████████████████████████████| 9.66G/9.66G [50:51<00:00, 3.40MB/s]


In [5]:
!unzip diabetic-retinopathy-level-detection

unzip:  cannot find either diabetic-retinopathy-level-detection or diabetic-retinopathy-level-detection.zip.


In [1]:
imageSize=[299,299]
testingpath = r"D:\VIT FOLDER\smartinternz-aiml\Diabetic Retinopathy\diabetic-retinopathy-level-detection\preprocessed dataset\preprocessed dataset\testing"
trainingpath = r"D:\VIT FOLDER\smartinternz-aiml\Diabetic Retinopathy\diabetic-retinopathy-level-detection\preprocessed dataset\preprocessed dataset\training"

In [2]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.xception import Xception,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
training_set = train_datagen.flow_from_directory("D:/VIT FOLDER/smartinternz-aiml/Diabetic Retinopathy/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/training",
                                                 target_size = (299,299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
testing_set = test_datagen.flow_from_directory("D:/VIT FOLDER/smartinternz-aiml/Diabetic Retinopathy/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/testing",
                                                 target_size = (299,299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [5]:
xception = Xception(input_shape = imageSize + [3],weights = 'imagenet',include_top = False)

In [6]:
for layer in xception.layers:
  layer.trainable = False

In [7]:
x = Flatten()(xception.output)

In [8]:
prediction = Dense(5,activation = 'softmax')(x)

In [9]:
model = Model(inputs = xception.input, outputs = prediction)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]'] 

                                                                                                  
 block4_sepconv2_act (Activ  (None, 37, 37, 728)          0         ['block4_sepconv1_bn[0][0]']  
 ation)                                                                                           
                                                                                                  
 block4_sepconv2 (Separable  (None, 37, 37, 728)          536536    ['block4_sepconv2_act[0][0]'] 
 Conv2D)                                                                                          
                                                                                                  
 block4_sepconv2_bn (BatchN  (None, 37, 37, 728)          2912      ['block4_sepconv2[0][0]']     
 ormalization)                                                                                    
                                                                                                  
 conv2d_2 

                                                                                                  
 block7_sepconv1 (Separable  (None, 19, 19, 728)          536536    ['block7_sepconv1_act[0][0]'] 
 Conv2D)                                                                                          
                                                                                                  
 block7_sepconv1_bn (BatchN  (None, 19, 19, 728)          2912      ['block7_sepconv1[0][0]']     
 ormalization)                                                                                    
                                                                                                  
 block7_sepconv2_act (Activ  (None, 19, 19, 728)          0         ['block7_sepconv1_bn[0][0]']  
 ation)                                                                                           
                                                                                                  
 block7_se

 ormalization)                                                                                    
                                                                                                  
 add_7 (Add)                 (None, 19, 19, 728)          0         ['block9_sepconv3_bn[0][0]',  
                                                                     'add_6[0][0]']               
                                                                                                  
 block10_sepconv1_act (Acti  (None, 19, 19, 728)          0         ['add_7[0][0]']               
 vation)                                                                                          
                                                                                                  
 block10_sepconv1 (Separabl  (None, 19, 19, 728)          536536    ['block10_sepconv1_act[0][0]']
 eConv2D)                                                                                         
          

 block12_sepconv3_act (Acti  (None, 19, 19, 728)          0         ['block12_sepconv2_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 block12_sepconv3 (Separabl  (None, 19, 19, 728)          536536    ['block12_sepconv3_act[0][0]']
 eConv2D)                                                                                         
                                                                                                  
 block12_sepconv3_bn (Batch  (None, 19, 19, 728)          2912      ['block12_sepconv3[0][0]']    
 Normalization)                                                                                   
                                                                                                  
 add_10 (Add)                (None, 19, 19, 728)          0         ['block12_sepconv3_bn[0][0]', 
          

In [12]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [13]:
r = model.fit_generator(
    training_set,
    validation_data = testing_set, 
    epochs = 20,
    steps_per_epoch = len(training_set ),
    validation_steps = len(testing_set)
)

C:\Users\HP\AppData\Local\Temp\ipykernel_13668\3649369830.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/20
115/115 [==============================] - 1226s 11s/step - loss: 4.4407 - accuracy: 0.6562 - val_loss: 4.1942 - val_accuracy: 0.6676
Epoch 2/20
115/115 [==============================] - 879s 8s/step - loss: 3.4793 - accuracy: 0.7171 - val_loss: 3.0072 - val_accuracy: 0.7684
Epoch 3/20
115/115 [==============================] - 745s 6s/step - loss: 3.5121 - accuracy: 0.7242 - val_loss: 2.7127 - val_accuracy: 0.7507
Epoch 4/20
115/115 [==============================] - 743s 6s/step - loss: 3.2569 - accuracy: 0.7540 - val_loss: 2.3483 - val_accuracy: 0.7684
Epoch 5/20
115/115 [==============================] - 736s 6s/step - loss: 3.1046 - accuracy: 0.7608 - val_loss: 4.7199 - val_accuracy: 0.6158
Epoch 6/20
115/115 [==============================] - 737s 6s/step - loss: 3.3806 - accuracy: 0.7622 - val_loss: 2.1419 - val_accuracy: 0.7929
Epoch 7/20
115/115 [==============================] - 735s 6s/step - loss: 2.8886 - accuracy: 0.7794 - val_loss: 2.8869 - val_accuracy: 0.76

In [14]:
model.save("New_xception_diabetic_retinopathy.h5")

C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
